In [4]:
from linkedin import Scraper

# selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
import pandas as pd
import time

# credentials
email = 'joseph.barbierdarnal@gmail.com'
with open('../credentials.txt', 'r') as file:
    password = file.read()

In [5]:
scraper = Scraper()
scraper.load_page("https://www.linkedin.com/")
scraper.accept_cookies()
scraper.login(email, password)

In [6]:
scraper.load_page(
    "https://www.linkedin.com/jobs/collections/recommended/",
    try_quitting_first=False
)
scraper.close_message()

scraper.enter_keywords('data science')
scraper.enter_location('European Economic Area')
scraper.press_enter_and_scroll()

scraper.scrap_jobs(driver)